testing clinet ans server (from my text
)


In [10]:
import flwr as fl
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

# Define the same neural network model as on the server
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(15, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

# Training function
def train(net, trainloader, epochs=20):
    net.train()
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    loss_fn = nn.BCELoss()

    for epoch in range(epochs):
        for data, target in trainloader:
            optimizer.zero_grad()
            output = net(data)
            loss = loss_fn(output.squeeze(), target)
            loss.backward()
            optimizer.step()

# Testing function
def test(net, valloader):
    net.eval()
    correct = 0
    total = 0
    loss_fn = nn.BCELoss()
    
    with torch.no_grad():
        for data, target in valloader:
            output = net(data)
            loss = loss_fn(output.squeeze(), target)
            predicted = (output.squeeze() > 0.5).float()
            total += target.size(0)
            correct += (predicted == target).sum().item()

    accuracy = correct / total
    print(f"Validation Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}")
    return loss.item(), accuracy

# Define dataset
class HeartDiseaseDataset(Dataset):
    def __init__(self, data, target):
        self.data = torch.tensor(data.values, dtype=torch.float32)
        self.target = torch.tensor(target.values, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.target[idx]

# Load and preprocess data
data = pd.read_csv('client_1.csv')  # Change file name for different clients
X = data.drop(columns=["TenYearCHD"])
y = data["TenYearCHD"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
train_dataset = HeartDiseaseDataset(pd.DataFrame(X_train), pd.Series(y_train))
test_dataset = HeartDiseaseDataset(pd.DataFrame(X_test), pd.Series(y_test))

trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the Flower Client
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, pid, net, trainloader, valloader):
        self.pid = pid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.pid}] Sending model parameters to server.")
        return [val.detach().cpu().numpy() for val in self.net.parameters()]

    def fit(self, parameters, config):
        server_round = config.get("server_round", 1)  # Prevent KeyError
        local_epochs = config.get("local_epochs", 60)  # Train for 20 epochs before sending updates

        print(f"[Client {self.pid}, round {server_round}] Training for {local_epochs} epochs...")
        
        set_parameters(self.net, parameters)  # Load global model parameters
        train(self.net, self.trainloader, epochs=local_epochs)  # Train for 20 epochs
        return self.get_parameters(config), len(self.trainloader.dataset), {}


    def evaluate(self, parameters, config):
        print(f"[Client {self.pid}] Evaluating model...")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return loss, len(self.valloader.dataset), {"accuracy": accuracy}

# Helper function to set model parameters
def set_parameters(net, parameters):
    for param, new_param in zip(net.parameters(), parameters):
        param.data = torch.tensor(new_param)
        
def train(net, trainloader, epochs):
    net.train()
    optimizer = optim.Adam(net.parameters(), lr=0.0005)
    loss_fn = nn.BCELoss()

    for epoch in range(epochs):
        epoch_loss = 0
        for data, target in trainloader:
            optimizer.zero_grad()
            output = net(data)
            loss = loss_fn(output.squeeze(), target)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(trainloader):.4f}")


if __name__ == "__main__":
    model = SimpleNN()
    client = FlowerClient(pid=1, net=model, trainloader=trainloader, valloader=valloader)

    fl.client.start_client(
        server_address="127.0.0.1:8081",
        client=client.to_client()  # Updated to latest Flower API
    )


	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        


INFO :      
INFO :      Received: get_parameters message ca479af3-0a7b-4297-871d-76d484af0420
INFO :      Sent reply


[Client 1] Sending model parameters to server.


INFO :      
INFO :      Received: train message 80e15dd9-4d9f-4fc6-a284-d389b3727826


[Client 1, round 1] Training for 60 epochs...
Epoch 1/60, Loss: 0.6852
Epoch 2/60, Loss: 0.6599
Epoch 3/60, Loss: 0.6407
Epoch 4/60, Loss: 0.6262
Epoch 5/60, Loss: 0.6164
Epoch 6/60, Loss: 0.6076
Epoch 7/60, Loss: 0.6001
Epoch 8/60, Loss: 0.5964
Epoch 9/60, Loss: 0.5893
Epoch 10/60, Loss: 0.5857
Epoch 11/60, Loss: 0.5805
Epoch 12/60, Loss: 0.5775
Epoch 13/60, Loss: 0.5717
Epoch 14/60, Loss: 0.5679
Epoch 15/60, Loss: 0.5632
Epoch 16/60, Loss: 0.5597
Epoch 17/60, Loss: 0.5551
Epoch 18/60, Loss: 0.5513
Epoch 19/60, Loss: 0.5482
Epoch 20/60, Loss: 0.5445
Epoch 21/60, Loss: 0.5404
Epoch 22/60, Loss: 0.5364
Epoch 23/60, Loss: 0.5330
Epoch 24/60, Loss: 0.5275
Epoch 25/60, Loss: 0.5247
Epoch 26/60, Loss: 0.5223
Epoch 27/60, Loss: 0.5182
Epoch 28/60, Loss: 0.5142
Epoch 29/60, Loss: 0.5130
Epoch 30/60, Loss: 0.5085
Epoch 31/60, Loss: 0.5059
Epoch 32/60, Loss: 0.5017
Epoch 33/60, Loss: 0.4994
Epoch 34/60, Loss: 0.4957
Epoch 35/60, Loss: 0.4942
Epoch 36/60, Loss: 0.4901
Epoch 37/60, Loss: 0.4874
E

...

INFO :      
INFO :      Received: evaluate message 6ed9eede-51da-49a4-b553-431c90a429db
INFO :      Sent reply
INFO :      
INFO :      Received: train message 762d2ccc-7373-40c2-a0a1-390f0790fd0c


[Client 1] Evaluating model...
Validation Loss: 0.5687, Accuracy: 0.7833
[Client 1, round 1] Training for 60 epochs...
Epoch 1/60, Loss: 0.3294
Epoch 2/60, Loss: 0.3003
Epoch 3/60, Loss: 0.2815
Epoch 4/60, Loss: 0.2711
Epoch 5/60, Loss: 0.2622
Epoch 6/60, Loss: 0.2537
Epoch 7/60, Loss: 0.2448
Epoch 8/60, Loss: 0.2411
Epoch 9/60, Loss: 0.2388
Epoch 10/60, Loss: 0.2313
Epoch 11/60, Loss: 0.2305
Epoch 12/60, Loss: 0.2240
Epoch 13/60, Loss: 0.2202
Epoch 14/60, Loss: 0.2175
Epoch 15/60, Loss: 0.2171
Epoch 16/60, Loss: 0.2130
Epoch 17/60, Loss: 0.2117
Epoch 18/60, Loss: 0.2061
Epoch 19/60, Loss: 0.2077
Epoch 20/60, Loss: 0.2060
Epoch 21/60, Loss: 0.2040
Epoch 22/60, Loss: 0.2020
Epoch 23/60, Loss: 0.1998
Epoch 24/60, Loss: 0.1973
Epoch 25/60, Loss: 0.1969
Epoch 26/60, Loss: 0.1939
Epoch 27/60, Loss: 0.1933
Epoch 28/60, Loss: 0.1944
Epoch 29/60, Loss: 0.1872
Epoch 30/60, Loss: 0.1875
Epoch 31/60, Loss: 0.1880
Epoch 32/60, Loss: 0.1866
Epoch 33/60, Loss: 0.1829
Epoch 34/60, Loss: 0.1782
Epoch 

INFO :      Sent reply
INFO :      
INFO :      Received: evaluate message 89bc4f3f-49e7-44e3-92ba-d8512a8f491b
INFO :      Sent reply
INFO :      
INFO :      Received: reconnect message c323bb87-f26b-46a5-8e44-ba5cb6f7995f


Epoch 60/60, Loss: 0.1530
[Client 1] Sending model parameters to server.
[Client 1] Evaluating model...
Validation Loss: 0.5948, Accuracy: 0.7917


INFO :      Disconnect and shut down
